In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline
import requests
import json
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
from collections import defaultdict

In [2]:
# read in csv file, remove first 3 rows and last 5 rows
tn_pop = pd.read_csv('../data/us_census_tn_pop_by_county_2010_2019.csv', skiprows=3, skipfooter=5, engine='python')
tn_pop.head(10)

,Unnamed: 0,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Tennessee,"6,346,105","6,346,276","6,355,311","6,399,291","6,453,898","6,494,340","6,541,223","6,591,170","6,646,010","6,708,799","6,771,631","6,829,174"
1,".Anderson County, Tennessee","75,129","75,082","75,098","75,209","75,225","75,299","75,157","75,456","75,528","76,056","76,287","76,978"
2,".Bedford County, Tennessee","45,058","45,057","45,078","45,247","45,254","45,565","46,251","46,948","47,442","48,211","49,146","49,713"
3,".Benton County, Tennessee","16,489","16,491","16,511","16,529","16,445","16,360","16,197","16,220","16,082","15,993","16,246","16,160"
4,".Bledsoe County, Tennessee","12,876","12,874","12,884","12,992","12,926","13,910","14,501","14,601","14,738","14,895","14,883","15,064"
5,".Blount County, Tennessee","123,010","123,098","123,199","123,664","124,024","124,874","125,847","126,954","128,264","129,999","131,331","133,088"
6,".Bradley County, Tennessee","98,963","98,926","99,083","99,798","101,049","101,778","102,771","103,774","104,390","105,421","107,050","108,110"
7,".Campbell County, Tennessee","40,716","40,723","40,735","40,688","40,532","40,247","39,893","39,772","39,784","39,791","39,795","39,842"
8,".Cannon County, Tennessee","13,801","13,813","13,797","13,724","13,789","13,700","13,572","13,740","13,876","14,157","14,440","14,678"
9,".Carroll County, Tennessee","28,522","28,486","28,454","28,547","28,667","28,606","28,411","27,982","27,901","27,807","27,975","27,767"


In [3]:
tn_pop.tail(15)

,Unnamed: 0,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
81,".Stewart County, Tennessee","13,324","13,313","13,348","13,223","13,309","13,277","13,210","13,187","13,190","13,422","13,619","13,715"
82,".Sullivan County, Tennessee","156,823","156,800","156,749","156,876","156,322","156,185","156,404","156,199","156,182","156,924","157,599","158,348"
83,".Sumner County, Tennessee","160,645","160,634","161,214","163,680","165,967","168,754","172,269","175,326","179,332","183,756","187,490","191,283"
84,".Tipton County, Tennessee","61,081","61,006","61,065","61,305","61,524","61,572","61,596","61,533","61,223","61,305","61,576","61,599"
85,".Trousdale County, Tennessee","7,870","7,864","7,874","7,815","7,794","7,810","7,998","8,051","9,955","10,831","11,033","11,284"
86,".Unicoi County, Tennessee","18,313","18,311","18,284","18,302","18,253","18,095","17,951","17,819","17,737","17,796","17,822","17,883"
87,".Union County, Tennessee","19,109","19,107","19,116","19,191","19,105","19,063","18,991","19,159","19,219","19,399","19,689","19,972"
88,".Van Buren County, Tennessee","5,548","5,558","5,569","5,555","5,653","5,574","5,696","5,695","5,721","5,747","5,763","5,872"
89,".Warren County, Tennessee","39,839","39,824","39,858","39,907","39,744","39,875","39,983","40,262","40,428","40,717","40,826","41,277"
90,".Washington County, Tennessee","122,979","123,063","123,384","123,887","124,873","125,397","125,745","126,137","127,247","127,603","128,661","129,375"


In [4]:
#drop all columns except for 2018, 2019
tn_pop = tn_pop.drop(columns = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017'], axis=1)

In [5]:
tn_pop.head()

,Unnamed: 0,Census,Estimates Base,2018,2019
0,Tennessee,"6,346,105","6,346,276","6,771,631","6,829,174"
1,".Anderson County, Tennessee","75,129","75,082","76,287","76,978"
2,".Bedford County, Tennessee","45,058","45,057","49,146","49,713"
3,".Benton County, Tennessee","16,489","16,491","16,246","16,160"
4,".Bledsoe County, Tennessee","12,876","12,874","14,883","15,064"


In [6]:
#rename 'Unnamed:0' column to 'County' 
tn_pop.columns = ['County', 'Census', 'Estimates_Base', '2018', '2019']
tn_pop.reset_index()
tn_pop.head()

,County,Census,Estimates_Base,2018,2019
0,Tennessee,"6,346,105","6,346,276","6,771,631","6,829,174"
1,".Anderson County, Tennessee","75,129","75,082","76,287","76,978"
2,".Bedford County, Tennessee","45,058","45,057","49,146","49,713"
3,".Benton County, Tennessee","16,489","16,491","16,246","16,160"
4,".Bledsoe County, Tennessee","12,876","12,874","14,883","15,064"


In [7]:
tn_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   County          96 non-null     object
 1   Census          96 non-null     object
 2   Estimates_Base  96 non-null     object
 3   2018            96 non-null     object
 4   2019            96 non-null     object
dtypes: object(5)
memory usage: 3.9+ KB


In [8]:
# remove '.' before each county name.  code below does not work.  also tried .strip and r'
# tn_pop['County'].str.replace('\.', '')
# tn_pop.head(10)

In [14]:
tn_pop['2019']=tn_pop['2019'].str.replace(',', '')
tn_pop

,County,Census,Estimates_Base,2018,2019
0,Tennessee,"6,346,105","6,346,276","6,771,631",6829174
1,".Anderson County, Tennessee","75,129","75,082","76,287",76978
2,".Bedford County, Tennessee","45,058","45,057","49,146",49713
3,".Benton County, Tennessee","16,489","16,491","16,246",16160
4,".Bledsoe County, Tennessee","12,876","12,874","14,883",15064
...,...,...,...,...,...
91,".Wayne County, Tennessee","17,021","17,025","16,635",16673
92,".Weakley County, Tennessee","35,021","35,015","33,413",33328
93,".White County, Tennessee","25,841","25,844","27,086",27345
94,".Williamson County, Tennessee","183,182","183,277","231,978",238412


In [15]:
tn_pop['2019']=tn_pop['2019'].astype(int)
tn_pop

,County,Census,Estimates_Base,2018,2019
0,Tennessee,"6,346,105","6,346,276","6,771,631",6829174
1,".Anderson County, Tennessee","75,129","75,082","76,287",76978
2,".Bedford County, Tennessee","45,058","45,057","49,146",49713
3,".Benton County, Tennessee","16,489","16,491","16,246",16160
4,".Bledsoe County, Tennessee","12,876","12,874","14,883",15064
...,...,...,...,...,...
91,".Wayne County, Tennessee","17,021","17,025","16,635",16673
92,".Weakley County, Tennessee","35,021","35,015","33,413",33328
93,".White County, Tennessee","25,841","25,844","27,086",27345
94,".Williamson County, Tennessee","183,182","183,277","231,978",238412


In [17]:
tn_pop.sort_values('2019', ascending = False)

,County,Census,Estimates_Base,2018,2019
0,Tennessee,"6,346,105","6,346,276","6,771,631",6829174
79,".Shelby County, Tennessee","927,644","927,682","936,365",937166
19,".Davidson County, Tennessee","626,681","626,558","690,516",694144
47,".Knox County, Tennessee","432,226","432,260","466,258",470313
33,".Hamilton County, Tennessee","336,463","336,477","364,293",367804
...,...,...,...,...,...
48,".Lake County, Tennessee","7,832","7,832","7,407",7016
34,".Hancock County, Tennessee","6,819","6,811","6,561",6620
64,".Moore County, Tennessee","6,362","6,342","6,457",6488
88,".Van Buren County, Tennessee","5,548","5,558","5,763",5872
